In [34]:
# Create a collection in customized setup mode
from pymilvus import (
    MilvusClient, DataType,Function,FunctionType
)

In [52]:

client = MilvusClient(
    uri="http://localhost:19530"
)

# Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)
# Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=1000,enable_analyzer=True)
# Define a sparse vector field to generate spare vectors with BM25
schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR)
schema.add_field(field_name="dense", datatype=DataType.FLOAT_VECTOR, dim=768)


{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000, 'enable_analyzer': True}}, {'name': 'sparse', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}, {'name': 'dense', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': True}

In [ ]:
client.drop_collection(
    collection_name="hybrid_search_collection"
) # chạy lệnh này để drop collection cũ trên milvus nếu tồn tại

In [53]:
# Define function to generate sparse vectors

bm25_function = Function(
    name="text_bm25_emb", # Function name
    input_field_names=["text"], # Name of the VARCHAR field containing raw text data
    output_field_names=["sparse"], # Name of the SPARSE_FLOAT_VECTOR field reserved to store generated embeddings
    function_type=FunctionType.BM25,
    

)

schema.add_function(bm25_function)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000, 'enable_analyzer': True}}, {'name': 'sparse', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>, 'is_function_output': True}, {'name': 'dense', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': True, 'functions': [{'name': 'text_bm25_emb', 'description': '', 'type': <FunctionType.BM25: 1>, 'input_field_names': ['text'], 'output_field_names': ['sparse'], 'params': {}}]}

In [54]:
from pymilvus import MilvusClient

# Prepare index parameters
index_params = client.prepare_index_params()

# Add indexes
index_params.add_index(
    field_name="dense",
    index_name="dense_index",
    index_type="IVF_FLAT",
    metric_type="IP",
    params={"nlist": 128},
)

index_params.add_index(
    field_name="sparse",
    index_name="sparse_index",
    index_type="SPARSE_INVERTED_INDEX",  # Index type for sparse vectors
    metric_type="BM25",  # Set to `BM25` when using function to generate sparse vectors
    params={"inverted_index_algo": "DAAT_MAXSCORE"},  # The ratio of small vector values to be dropped during indexing
)

In [55]:
from pymilvus import MilvusClient

client.create_collection(
    collection_name="hybrid_search_collection",
    schema=schema,
    index_params=index_params
)

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from underthesea import sent_tokenize
from langchain_community.document_loaders import TextLoader
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [59]:


def load_and_split_txt():
    # Bước 1: Đọc nội dung từ file TXT
    loader = TextLoader('ictu.txt',encoding='utf-8')
    documents = loader.load()

    # Bước 2: Ghép nội dung các phần lại
    full_text = "\n".join([doc.page_content for doc in documents])

    # Bước 3: Tách câu sử dụng underthesea (cho tiếng Việt)
    sentences = sent_tokenize(full_text)

    return sentences

In [60]:
chunks = load_and_split_txt()

In [61]:
chunks

['Khoa Công nghệ thông tin\n1.',
 'Lịch sử hình thành và phát triển\nKhoa Công nghệ thông tin thuộc Trường Đại học Công nghệ Thông tin và Truyền thông tiền thân có 04 bộ môn: Khoa học Máy tính, Công nghệ Phần mềm, Kỹ thuật Máy tính và Mạng máy tính & các Hệ thống Thông tin – là đơn vị chuyên môn chủ chốt của Khoa Công nghệ thông tin thuộc ĐHTN trong những năm đầu mới thành lập.Với đội ngũ cán bộ giảng viên trẻ, sáng tạo và nhiệt huyết, các Bộ môn đã có những đóng góp không nhỏ trong sự phát triển chung của nhà trường.',
 'Ngày 11/08/2011 Khoa Công nghệ thông tin thuộc Trường Đại học Công nghệ Thông tin và Truyền thông được thành lập theo Quyết định số 802/QĐ-ĐHTN của Giám đốc Đại học Thái nguyên.',
 'Theo Quyết định, Khoa có 04 Bộ môn: Khoa học Máy tính, Công nghệ Phần mềm, Hệ Thống thông Tin, Mạng & Truyền thông và bộ phận Văn phòng.',
 '2.Chức năng, nhiệm vụ: Khoa có chức năng, nhiệm vụ đào tạo nguồn nhân lực chất lượng về lĩnh vực Công nghệ thông tin; nghiên cứu khoa học và chuyển g

In [62]:
sbert_model = SentenceTransformer.load("./models/vietnamese-sbert")

In [63]:
def emb_text(text):
    # Tạo embedding từ SBERT
    embedding = sbert_model.encode(text).tolist()
    return embedding

In [64]:
data = []
for i, line in enumerate(tqdm(chunks, desc="Creating embeddings")):
    data.append({"id": i,"text": line ,"dense": emb_text(line) })
client.insert(collection_name='hybrid_search_collection', data=data)

Creating embeddings: 100%|██████████| 14/14 [00:01<00:00, 11.43it/s]


{'insert_count': 14, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'cost': 0}

In [66]:
querry="Lãnh đạo Bộ môn/Khoa qua các thời kỳ ?"

In [67]:
from pymilvus import AnnSearchRequest

search_param_1 = {
    "data": [emb_text(querry)],
    "anns_field": "dense",
    "param": {
        "metric_type": "IP",
        "params": {"nprobe": 10}
    },
    "limit": 2
}
request_1 = AnnSearchRequest(**search_param_1)

search_param_2 = {
    "data": [querry],
    "anns_field": "sparse",
    "param": {
        "metric_type": "BM25",
    },
    "limit": 2
}
request_2 = AnnSearchRequest(**search_param_2)

reqs = [request_1, request_2]

In [ ]:
# from pymilvus import WeightedRanker
# bạn có thể dùng thằng WeightedRanker để tùy chỉnh xem bạn muốn
# phần nào quan trọng hơn ngữ nghĩa hay là khớp từ
# ranker = WeightedRanker(0.8, 0.3) 

In [ ]:
from pymilvus import RRFRanker
# ở đây sử dụng RRF để nó tự cân bằng giữa sparse và dense
ranker = RRFRanker(100)

In [70]:
from pymilvus import MilvusClient

res = client.hybrid_search(
    collection_name="hybrid_search_collection",
    reqs=reqs,
    ranker=ranker,
    limit=2,
    output_fields=["text"]
)
for hits in res:
    print("TopK results:")
    for hit in hits:
        print(hit)

TopK results:
{'id': 9, 'distance': 0.019801979884505272, 'entity': {'text': '4.Lãnh đạo Bộ môn/Khoa qua các thời kỳ\nTrưởng Bộ môn/Khoa: Giai đoạn 2004-2011 (PGS.TS Đoàn Văn Ban; PGS.TS Đặng Văn Đức; GS.TS Vũ Đức Thi; PGS.TS Nguyễn Văn Tam; PGS.TS Phạm Việt Bình; TS Lê Quang Minh); Giai đoạn 2016-2021 (TS. Nguyễn Hải Minh) giai đoạn 2021-2026 (TS. Nguyễn Hải Minh; TS. Quách Xuân Trưởng).'}}
{'id': 4, 'distance': 0.009803921915590763, 'entity': {'text': '2.Chức năng, nhiệm vụ: Khoa có chức năng, nhiệm vụ đào tạo nguồn nhân lực chất lượng về lĩnh vực Công nghệ thông tin; nghiên cứu khoa học và chuyển giao công nghệ cho các tỉnh trung du miền núi phía Bắc và cả nước.'}}
